In [1]:
from tensorflow.keras import backend as K
import os
import numpy as np
import random
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, UpSampling2D, Dropout,BatchNormalization,ZeroPadding2D,add, Flatten,Activation,AveragePooling2D,Dense
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,CSVLogger,ReduceLROnPlateau
from tensorflow.keras.layers import Lambda
from tensorflow.keras.preprocessing.image import img_to_array,ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model,to_categorical
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps, ImageFile
import time
from all_model import all_model
from utils import *
# from historycal import LossHistory
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
hdf5_path = "/home/nmic/input/nas5918/HJtest2-tf1/data/train_5000.hdf5"
fd = h5py.File(hdf5_path)
images,labels=fd['image'],fd['label']
images,labels=np.asarray(fd['image']),np.asarray(fd['label'])
print(images.shape,labels.shape)
fd.close()

# classes = np.unique(labels)  #标签类别
# class_weight = compute_class_weight('balanced', classes, labels.ravel())
# print(class_weight)
# class_weight=[0.27993096,1.59620928,32.11132344,1.81175424,0.842855,31.57588407]

n_label=6 #耕地=0，林地=1，草地=2，水域=3，居民地=4，未利用=5
img,msk = get_normalized_patches(images[:2000],labels[:2000],n_label)

(5000, 256, 256, 3) (5000, 256, 256)
(2000, 256, 256, 3) (2000, 256, 256, 6)


In [4]:
del images,labels
xtrain,xtest,ytrain,ytest=train_test_split(img,msk,test_size=0.2,random_state=42)
del img,msk
print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

(1600, 256, 256, 3) (400, 256, 256, 3) (1600, 256, 256, 6) (400, 256, 256, 6)


In [5]:
plot_func(xtrain[40:, :, :, :], ytrain[40:, :, :, 0])

<Figure size 1800x360 with 20 Axes>

<Figure size 1800x360 with 20 Axes>

In [6]:
loss='categorical_crossentropy'#'categorical_crossentropy'#'binary_crossentropy'FocalLoss
optimizer=Adam()#Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#Adam()#SGD(),Adamax(),Adadelta()R
loss_weights=''
metrics=['accuracy']
input_height=256
input_width=256
batch_size=10
epoch=20
nclass=n_label
nchannel=xtrain.shape[-1]
num_train=xtrain.shape[0]
num_val=xtest.shape[0]
ATM=all_model(loss,loss_weights,optimizer,metrics,input_height,input_width,nclass,nchannel)

In [7]:
nickname='UNET_VGG'
model=ATM.UNET_VGG()#FCN8,UNET_VGG,UNET_MINI,SQUEESE_UNET,DEEPLABV3plus,CBRRNET,GFCpsp_xception

2022-04-16 14:50:08.391509: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-16 14:50:08.948681: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30981 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


In [8]:
begin_time = time.time()
checkpoint_DIR = "./checkpoint/"
if not os.path.exists(checkpoint_DIR):
    os.makedirs(checkpoint_DIR)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_DIR + nickname + "-{epoch:02d}e-val_loss{val_loss:2f}.hdf5", monitor="val_loss",save_best_only=True,mode='auto')
csvlogger = CSVLogger(filename=checkpoint_DIR +nickname +'-' +str(epoch) +'-log.csv', separator=',', append=False)
# result=model.fit_generator(G1,verbose=1,steps_per_epoch=num_train/batch_size,epochs=epoch,validation_data=G2,validation_steps=num_val/batch_size,callbacks=[model_checkpoint,histories],shuffle=True )

result = model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epoch,verbose=1,shuffle=True,validation_data=( xtest, ytest),callbacks=[ model_checkpoint, csvlogger])
end_time = time.time()
print('time lapsing {0} s \n'.format(end_time - begin_time))


Epoch 1/20


2022-04-16 14:50:19.234930: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-04-16 14:50:19.669429: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


160/160 [==============================] - 29s 135ms/step - loss: 1.1737 - accuracy: 0.6022 - val_loss: 1.1307 - val_accuracy: 0.5683
Epoch 2/20
160/160 [==============================] - 22s 140ms/step - loss: 1.0648 - accuracy: 0.6037 - val_loss: 1.1383 - val_accuracy: 0.5684
Epoch 3/20
160/160 [==============================] - 22s 141ms/step - loss: 1.0523 - accuracy: 0.6134 - val_loss: 1.0428 - val_accuracy: 0.6019
Epoch 4/20
160/160 [==============================] - 21s 133ms/step - loss: 1.0135 - accuracy: 0.6275 - val_loss: 1.1161 - val_accuracy: 0.5951
Epoch 5/20
160/160 [==============================] - 22s 136ms/step - loss: 0.9894 - accuracy: 0.6406 - val_loss: 0.9820 - val_accuracy: 0.6351
Epoch 6/20
160/160 [==============================] - 20s 127ms/step - loss: 0.9961 - accuracy: 0.6370 - val_loss: 1.0103 - val_accuracy: 0.6163
Epoch 7/20
160/160 [==============================] - 22s 138ms/step - loss: 0.9721 - accuracy: 0.6448 - val_loss: 0.9704 - val_accuracy: 0.6

In [9]:
def plot_fig(H,outdir):
    N=len(result.history['loss'])
    plt.style.use("ggplot")
    plt.figure(figsize=(10,6))
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.ylim(0,1)

    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(outdir)
plot_fig(result,nickname+"_Loss_Acc_epoch.png")

<Figure size 720x432 with 1 Axes>

In [2]:
hdf5_path = '/home/nmic/input/nas5918/HJtest2-tf1/data/val-data-500.hdf5' 
fd = h5py.File(hdf5_path)
fd.keys()
xval,yval=np.asarray(fd['xval']),np.asarray(fd['yval'])
fd.close

<bound method File.close of <HDF5 file "val-data-500.hdf5" (mode r)>>

In [3]:
model=load_model('/home/nmic/project/checkpoint/UNET_VGG-19e-val_loss0.918906.hdf5')

2022-04-16 15:06:50.969634: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-16 15:06:51.553735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30981 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


In [4]:
i=0
n_label=6
nickname='unet_vgg'
result=model.predict(xval/255.0,)
pred=np.argmax(result,axis=-1)
newy=label_hot(yval,n_label)
val_plot_func(xval[i:i+20]/255.0,result[i:i+20,:,:,4],newy[i:i+20,:,:,4])
i+=30
#耕地=0，林地=1，草地=2，水域=3，居民地=4，未利用=5

2022-04-16 15:06:55.311978: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-04-16 15:06:55.781083: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


<Figure size 1800x360 with 20 Axes>

<Figure size 1800x360 with 20 Axes>

<Figure size 1800x360 with 20 Axes>

In [5]:
cal_table=val_cal(yval, pred, './'+nickname+'_result.csv', if_show=True)

          accuracy  precision    recall  F1-score       iou        oa  \
crop      0.702031   0.881798  0.698143  0.779296  0.638399  0.662983   
tree      0.913546   0.294729  0.676472  0.410576  0.258317  0.662983   
grass     0.993365   0.000000       NaN  0.000000  0.000000  0.662983   
water     0.900902   0.026655  0.640452  0.051179  0.026262  0.662983   
building  0.821745   0.551791  0.526484  0.538841  0.368776  0.662983   
others    0.994376   0.000000       NaN  0.000000  0.000000  0.662983   
mean      0.887661   0.292495       NaN  0.296649  0.215292  0.662983   

              miou     fwiou  
crop      0.215292  0.565594  
tree      0.215292  0.565594  
grass     0.215292  0.565594  
water     0.215292  0.565594  
building  0.215292  0.565594  
others    0.215292  0.565594  
mean      0.215292  0.565594  


/home/nmic/project/utils.py:70: RuntimeWarning: invalid value encountered in true_divide
  recall = TP/(TP+FN)
